# Etapa 2

Uma vez gereda a tabela dos genes e subtipos moleculares, executa o k-fold para obter os valores SHAP de cada gene.

In [4]:
from scipy.stats import chi2_contingency
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap

## 2.1 - Carrega tabela com genes e subtipos moleculares

In [5]:
df_final = pd.read_csv('outputs/gene_subtype_table.csv')
df_final.head()

,Unnamed: 0,Sample ID,Subtype,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,...,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,ago.1,ago.2,ago.3,ago.4,pk
0,0,TCGA-3M-AB46,cin,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,TCGA-3M-AB47,gs,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,TCGA-B7-5816,msi,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,TCGA-B7-5818,ebv,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,TCGA-B7-A5TI,msi,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2.2 - Separando grupo teste hold out

Verificando se a lista dos casos de test utilizados como teste para o ensemble estão no conjunto dos casos restantes

In [6]:
test_list = ['TCGA-D7-A6EY', 'TCGA-CD-A48C', 'TCGA-D7-5578', 'TCGA-CG-4469', 'TCGA-HU-A4GD', 'TCGA-BR-8371', 'TCGA-CD-A4MJ', 'TCGA-BR-8291', 'TCGA-RD-A8N4', 'TCGA-BR-6801', 'TCGA-CG-4462', 'TCGA-RD-A8N2', 'TCGA-D7-6822', 'TCGA-CD-8530', 'TCGA-D7-A748', 'TCGA-HU-A4H3', 'TCGA-HF-A5NB', 'TCGA-BR-7715', 'TCGA-D7-A6EZ', 'TCGA-B7-A5TN', 'TCGA-HF-7136', 'TCGA-CG-4444', 'TCGA-D7-6518', 'TCGA-BR-7723', 'TCGA-RD-A7BS', 'TCGA-D7-6521', 'TCGA-BR-6707', 'TCGA-BR-A4IZ', 'TCGA-D7-A747', 'TCGA-VQ-A8PD', 'TCGA-VQ-A923', 'TCGA-D7-A6F0', 'TCGA-HU-A4H8', 'TCGA-VQ-A8PF', 'TCGA-HU-A4GF', 'TCGA-BR-7196', 'TCGA-BR-6852', 'TCGA-BR-7959', 'TCGA-VQ-A8P8', 'TCGA-BR-7957', 'TCGA-BR-8590', 'TCGA-MX-A5UJ', 'TCGA-BR-8679', 'TCGA-BR-6566', 'TCGA-D7-8572', 'TCGA-VQ-AA69', 'TCGA-HF-7132', 'TCGA-CG-4443', 'TCGA-VQ-A8PB', 'TCGA-BR-4279', 'TCGA-CD-8535', 'TCGA-CD-A486', 'TCGA-VQ-A8PX', 'TCGA-HU-8604', 'TCGA-MX-A5UG', 'TCGA-BR-4357', 'TCGA-HU-A4G9', 'TCGA-BR-A4QI', 'TCGA-HU-A4GU', 'TCGA-BR-A452', 'TCGA-D7-6519', 'TCGA-D7-8570', 'TCGA-HU-A4H6', 'TCGA-CG-5720', 'TCGA-BR-7901', 'TCGA-BR-6706', 'TCGA-BR-8687', 'TCGA-VQ-A928', 'TCGA-BR-8081', 'TCGA-3M-AB47', 'TCGA-CG-4436', 'TCGA-BR-8058', 'TCGA-VQ-A91W', 'TCGA-BR-4361', 'TCGA-BR-4370', 'TCGA-D7-A6EX', 'TCGA-BR-7707', 'TCGA-CG-4477', 'TCGA-CG-5726', 'TCGA-BR-4253', 'TCGA-D7-A6F2', 'TCGA-HU-8249']

In [7]:
print("Casos que estão na lista dos casos usados para o teste do ensemble mas não estão nos casos restantes das tabelas mesclada de subtipos e genes")
[x for x in test_list if x not in df_final['Sample ID'].unique()]

Casos que estão na lista dos casos usados para o teste do ensemble mas não estão nos casos restantes das tabelas mesclada de subtipos e genes


['TCGA-RD-A8N2']

In [8]:
test_list.remove('TCGA-RD-A8N2')
print(f"Quantidade de casos que serão separados para o dataset de teste: {len(test_list)}")

Quantidade de casos que serão separados para o dataset de teste: 81


In [9]:
def create_data_set(df_final):
    test = df_final['Sample ID'].isin(test_list)
    test_df = df_final[test]
    print(f'Dimensções do data frame de teste: {test_df.shape}')

    train_val = ~test
    train_val_df = df_final[train_val]
    print(f'Dimensções do data frame de de teste e validação: {train_val_df.shape}')
    
    return train_val_df, test_df

In [10]:
train_val_df, test_df = create_data_set(df_final)

Dimensções do data frame de teste: (81, 18294)
Dimensções do data frame de de teste e validação: (290, 18294)


In [11]:
print("Distribuição no dataset de teste:")
test_df['Subtype'].value_counts()

Distribuição no dataset de teste:


Subtype
cin    41
msi    18
gs     12
ebv    10
Name: count, dtype: int64

## 2.3 - Separando dados para o treinamento

In [12]:
def X_y_df_split(train_val_df, test_df):
    # Ler seus dados:
    X_train_val = train_val_df.drop(["Subtype", "Sample ID"], axis=1)
    y_train_val = train_val_df["Subtype"] #.values

    X_test      = test_df.drop(["Subtype", "Sample ID"], axis=1)
    y_test      = test_df["Subtype"] #.values
    sample_ids  = test_df["Sample ID"] # .values
    
    return X_train_val, y_train_val, X_test, y_test, sample_ids

In [13]:
X_train_val, y_train_val, X_test, y_test, sample_ids = X_y_df_split(train_val_df, test_df)

## 2.4 - Treinando Modelos e Obtendo Valores SHAP

In [44]:
def create_result_table(y_true, y_pred, y_prob, rf_model, fold, sample_ids):
    
    results_df = pd.DataFrame({
            'Sample ID': sample_ids,   # 1. Copiando apenas a coluna Sample ID
            'true_label': y_true.values, # [x.split('_')[1].lower() for x in y_true.values],   # 2. Garantindo que y_test esteja como array alinhado
            'predicted_label': y_pred, #[x.split('_')[1].lower() for x in y_pred],   # 4. Previsões
            'predicted_probability': [max(prob) for prob in y_prob],    # Probabilidade da classe prevista
            'probability_vector': list(y_prob)   # Vetor de probabilidades
        })

    # print(f'results_df: {results_df["probability_vector"]}')

    # 5. Se quiser o probability vector completo:
    # classes = rf_model.classes_
    # results_df['probability_vector'] = [dict(zip(classes, prob)) for prob in y_prob]

    results_df.to_csv(f'outputs/kfold_random_fores_restuls/rf_test_df_fold{fold}_restuls.csv')


In [45]:
def rf_kfold_exe(X_train_val, y_train_val, train_val_df):
    """
    Executa k-fold com Random Forest, otimização de hiperparâmetros, métricas, classification report,
    índices de validação por fold e tabelas de importância SHAP (geral e por subtipo).

    Args:
        X_train_val (pd.DataFrame): Features (e.g., genes do painel).
        y_train_val (pd.Series): Rótulos (cin, ebv, gs, msi).
        train_val_df (pd.DataFrame): DataFrame com 'Sample ID' para create_result_table.

    Returns:
        models: Lista de modelos RF treinados por fold.
        accuracies, f1_scores, precisions, recalls: Listas de métricas por fold.
        reports: Lista de classification reports por fold.
    """
    # Defina os parâmetros para otimização do Random Forest
    param_dist = {
        'n_estimators': [100, 200, 300, 400, 500],
        'max_features': ['sqrt', 'log2', None],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    }

    # K-fold
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    # Para coletar resultados
    shap_values_folds = []
    models = []
    best_params_per_fold = []
    val_indices_per_fold = []
    accuracies = []
    f1_scores = []
    precisions = []
    recalls = []
    reports = []
    
    results_dict = {}

    # Iniciar RandomizedSearchCV
    for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train_val, y_train_val)):
        X_tr, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
        y_tr, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]

        # Salvar índices de validação
        val_indices_per_fold.append(val_idx)

        # Instanciar modelo
        rf = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')

        # Otimizar hiperparâmetros
        random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                          n_iter=10, cv=3, n_jobs=-1, random_state=42)
        random_search.fit(X_tr, y_tr)

        # Salvar melhores parâmetros
        best_params_per_fold.append(random_search.best_params_)
        best_rf_model = random_search.best_estimator_
        models.append(best_rf_model)

        # Previsões
        y_val_pred = best_rf_model.predict(X_val)
        y_val_prob = best_rf_model.predict_proba(X_val)

        # Tabela de resultados (função externa fornecida por você)
        sample_ids = train_val_df['Sample ID'].iloc[val_idx]
        create_result_table(y_val, y_val_pred, y_val_prob, best_rf_model, fold_idx, sample_ids)

        # Classification report
        report = classification_report(y_val, y_val_pred, target_names=best_rf_model.classes_, output_dict=True, zero_division=0)
        report_df = pd.DataFrame(report).transpose().round(2)
        report_df.to_csv(f"outputs/cr_val_fold{fold_idx}.csv")
        reports.append(report_df)
        print(f"\nClassification Report fold{fold_idx}:")
        print(report_df)

        # SHAP para o fold
        explainer = shap.TreeExplainer(best_rf_model)
        shap_vals = explainer.shap_values(X_val, check_additivity=False)
        shap_values_folds.append(shap_vals)

        # Calcular métricas
        accuracies.append(accuracy_score(y_val, y_val_pred))
        f1_scores.append(f1_score(y_val, y_val_pred, average='macro'))
        precisions.append(precision_score(y_val, y_val_pred, average='macro', zero_division=0))
        recalls.append(recall_score(y_val, y_val_pred, average='macro', zero_division=0))

    # Calcular métricas médias
    print("Resultados dos modelos dos 10-fold com Hyperp. Otimiz. no conjunto de validação")
    print(f"Média da Acurácia: {np.mean(accuracies):.2f} +- {np.std(accuracies):.2f}")
    print(f"Média do F1-Score: {np.mean(f1_scores):.2f} +- {np.std(f1_scores):.2f}")
    print(f"Média da Precisão: {np.mean(precisions):.2f} +- {np.std(precisions):.2f}")
    print(f"Média do Recall: {np.mean(recalls):.2f} +- {np.std(recalls):.2f}")

    # Salvar métricas médias
    metrics_df = pd.DataFrame({
        'Acurácia Média': [np.mean(accuracies)],
        'Acurácia Std (+-)': [np.std(accuracies)],
        'F1-Score Médio': [np.mean(f1_scores)],
        'F1-Score Std (+-)': [np.std(f1_scores)],
        'Precisão Média': [np.mean(precisions)],
        'Precisão Std (+-)': [np.std(precisions)],
        'Recall Médio': [np.mean(recalls)],
        'Recall Std': [np.std(recalls)]
    })
    metrics_df.to_csv('outputs/metricas_media_kfold_hyper_10.csv', index=False)

    # Calcular SHAP médio por subtipo
    classes = list(models[0].classes_)  # ['cin', 'ebv', 'gs', 'msi']
    shap_values_folds_mean_per_class = {cls: [] for cls in classes}

    for shap_vals in shap_values_folds:
        for cls_idx, cls in enumerate(classes):
            # Média absoluta de SHAP por feature e classe no fold
            shap_mean_cls = np.abs(shap_vals[cls_idx]).mean(axis=0)
            shap_values_folds_mean_per_class[cls].append(shap_mean_cls)

    # Gerar tabela de importância SHAP por subtipo
    shap_importance_per_class = []
    for cls in classes:
        # Média dos SHAPs por classe sobre os folds
        mean_shap_cls = np.mean(shap_values_folds_mean_per_class[cls], axis=0)
        # Ordenar por magnitude decrescente
        top_indices = np.argsort(mean_shap_cls)[::-1]
        # Criar DataFrame para a classe
        class_df = pd.DataFrame({
            'Gene': X_train_val.columns[top_indices],
            f'SHAP Importance ({cls})': mean_shap_cls[top_indices]
        })
        class_df['Subtipo'] = cls
        shap_importance_per_class.append(class_df)

    # Concatenar e salvar tabela de importância por subtipo
    shap_importance_df = pd.concat(shap_importance_per_class, ignore_index=True)
    shap_importance_df.to_csv('outputs/gene_importance_shap_per_subtype.csv', index=False)
    print("\nTabela de Importância SHAP por Subtipo salva em 'gene_importance_shap_per_subtype.csv'")
    print(shap_importance_df.head(20))  # Mostrar top 5 por subtipo

    # Tabela de importância SHAP geral (já existente)
    shap_values_folds_mean = [np.mean([np.abs(s).mean(axis=0) for s in shap_vals], axis=0) for shap_vals in shap_values_folds]
    mean_shap_folds = np.mean(shap_values_folds_mean, axis=0)
    top_indices = np.argsort(mean_shap_folds)[::-1]
    gene_importance_df = pd.DataFrame({
        'Gene': X_train_val.columns[top_indices],
        'SHAP Importance': mean_shap_folds[top_indices]
    })
    gene_importance_df.to_csv('outputs/gene_importance_shap.csv', index=False)
    print("\nTabela de Importância SHAP Geral salva em 'gene_importance_shap.csv'")

    # Salvar índices de validação
    val_indices_df = pd.DataFrame({
        'Fold': list(range(10)),
        'Val Indices': [list(indices) for indices in val_indices_per_fold]
    })
    val_indices_df.to_csv('outputs/val_indices_per_fold.csv', index=False)
    print("\nÍndices de Validação salvos em 'val_indices_per_fold.csv'")
    
    results_dict = {
        'shap_values_folds': shap_values_folds,
        'models': models,
        'best_params_per_fold': best_params_per_fold,
        'val_indices_per_fold': val_indices_per_fold,
        'accuracies': accuracies,
        'f1_scores': f1_scores,
        'precisions': precisions,
        'recalls': recalls,
        'reports': reports,
        'val_indices_df': val_indices_df, 
        'shap_values_folds_mean_per_class': shap_values_folds_mean_per_class,
        'metrics_df': metrics_df
    }
    

    return results_dict

In [46]:
result_dict = rf_kfold_exe(X_train_val, y_train_val, train_val_df)


Classification Report fold0:
              precision  recall  f1-score  support
cin                0.82    0.74      0.78    19.00
ebv                0.00    0.00      0.00     2.00
gs                 0.14    0.33      0.20     3.00
msi                1.00    1.00      1.00     5.00
accuracy           0.69    0.69      0.69     0.69
macro avg          0.49    0.52      0.49    29.00
weighted avg       0.73    0.69      0.70    29.00

Classification Report fold1:
              precision  recall  f1-score  support
cin                0.83    0.79      0.81    19.00
ebv                0.00    0.00      0.00     2.00
gs                 0.33    0.67      0.44     3.00
msi                1.00    1.00      1.00     5.00
accuracy           0.76    0.76      0.76     0.76
macro avg          0.54    0.61      0.56    29.00
weighted avg       0.75    0.76      0.75    29.00

Classification Report fold2:
              precision  recall  f1-score  support
cin                0.94    0.89      0.91  